In [ ]:
# [Cell 1] 환경 설정
# Kaggle 환경에서 NumPy 버전 충돌을 방지하고 YOLO 최신 버전을 설치합니다.
!pip install "numpy<2.0" "scipy<1.13" ultralytics scikit-learn opencv-python-headless -U -q

import os
import cv2
import yaml
import torch
import ultralytics
from ultralytics import YOLO
import matplotlib.pyplot as plt
from IPython.display import display, Image

print(f"✅ Setup Complete")
print(f"Ultralytics: {ultralytics.__version__}")
print(f"Torch: {torch.__version__}")
!nvidia-smi

In [ ]:
# [Cell 2] 데이터셋 및 모델 다운로드
if os.path.exists('MCA-Appendix'):
    !rm -rf MCA-Appendix

!git clone https://github.com/ICT-Top-Bottom/MCA-Appendix.git

# 경로 변수 설정
REPO_DIR = '/kaggle/working/MCA-Appendix'
DATA_YAML = f'{REPO_DIR}/data.yaml'
BEST_PT = f'{REPO_DIR}/best.pt'

print(f"\n✅ Repository Cloned at: {REPO_DIR}")

# data.yaml 경로 수정 (절대 경로로 안전하게)
with open(DATA_YAML, 'r') as f:
    data_config = yaml.safe_load(f)

data_config['path'] = f'{REPO_DIR}/dataset'
data_config['train'] = 'train/images'
data_config['val'] = 'valid/images'
data_config['test'] = 'test/images'

with open(DATA_YAML, 'w') as f:
    yaml.dump(data_config, f)

print("✅ data.yaml configured for Kaggle environment.")

In [ ]:
# [Cell 3] 모델 학습 (Training)
# 논문에 제안된 설정으로 학습을 재현합니다.
# 학습을 건너뛰고 바로 추론하려면 이 셀은 실행하지 않아도 됩니다.

print("🚀 Training Started...")
model = YOLO('yolo11s-seg.pt')

results = model.train(
    data=DATA_YAML,
    
    # --- System Config (Kaggle T4 x2 Optimized) ---
    device=[0, 1],
    batch=8,                # 1280px 메모리 최적값 (OOM 방지)
    epochs=200,
    patience=40,
    
    # --- Proposed Method Config ---
    imgsz=1280,             # High-Resolution (핵심)
    multi_scale=True,
    optimizer='AdamW',
    lr0=0.001,
    
    # --- Segmentation Tuning ---
    mask_ratio=4,           # Wireframe 뭉개기 (성능 핵심)
    overlap_mask=True,
    close_mosaic=10,
    
    # --- Loss & Augmentation ---
    cls=0.5,
    box=7.5,
    mixup=0.15,             # Combined Cart 학습용
    mosaic=1.0,
    degrees=15.0,
    
    # --- Reproducibility ---
    seed=42,
    deterministic=True,
    
    project='runs/train',
    name='MCA-CartSeg_Reproduction',
    save=True,
    plots=True
)
print("✅ Training Finished! Best model saved at runs/train/HR-CartSeg_Reproduction/weights/best.pt")

In [ ]:
# [Cell 4] 추론 실행 및 시각화 (Inference)
# 학습된 모델이 있으면 사용하고, 없으면 GitHub에서 받은 best.pt를 사용합니다.

# 1. 모델 로드
if os.path.exists('runs/train/MCA-CartSeg_Reproduction/weights/best.pt'):
    model_path = 'runs/train/MCA-CartSeg_Reproduction/weights/best.pt'
    print(f"✅ Loading Trained Model: {model_path}")
else:
    model_path = BEST_PT
    print(f"✅ Loading Pre-trained Model (Repo): {model_path}")

model = YOLO(model_path)

# 2. 테스트 이미지 추론
test_img_path = f'{REPO_DIR}/dataset/test/images/test1.jpg' # 테스트 이미지 예시

if os.path.exists(test_img_path):
    print(f"🔍 Running Inference on: {test_img_path}")
    results = model.predict(test_img_path, imgsz=1280, conf=0.25)
    
    # 결과 시각화
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(results[0].plot(), cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title(f"Inference Result (Conf > 0.25)")
    plt.show()
else:
    print("❌ Test image not found. Please check dataset path.")

In [ ]:
# [Cell 5] 동영상 파일 처리 (Optional)
# 동영상 파일을 업로드하거나 경로를 지정하여 실행하세요.

def run_inference_on_video(video_path, model_path):
    print(f"🎬 Processing Video: {video_path}...")
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("❌ Error: Could not open video.")
        return

    # 비디오 정보
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # 결과 저장용 Writer
    output_path = 'output_result.mp4'
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    
    frame_count = 0
    model = YOLO(model_path)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
        
        # 추론 (Retina Masks로 고화질 유지)
        results = model.predict(frame, conf=0.25, retina_masks=True, verbose=False)
        annotated_frame = results[0].plot() 
        
        out.write(annotated_frame)
        frame_count += 1
        
        if frame_count % 30 == 0:
            print(f"Processing frame {frame_count}...")
            
    cap.release()
    out.release()
    print(f"✅ Video processing completed! Saved to {output_path}")

# 사용 예시 (파일 경로 수정 필요)
# run_inference_on_video('/kaggle/input/test-video.mp4', model_path)